In [ ]:
# run this cell to install requirements
!pip install -r oddsgym[examples] -i https://pypi.python.org/simple

# 3 Way Soccer

In this notebook we will use the ThreeWaySoccerEnv to check various betting strategies.
For data, we will use the free data provided by www.football-data.co.uk.
Specifically for this notebook we will use the English Premier League.

## Getting the data

The site keeps the data in CSV files, which we can easily load with pandas as such:

In [1]:
import pandas as pd
raw_odds_data = pd.concat([pd.read_csv('http://www.football-data.co.uk/mmz4281/{}{}/E0.csv'.format(i, i +1)) for i in range(10, 17)])
raw_odds_data['Date'] = pd.to_datetime(raw_odds_data['Date'], dayfirst=True)

/Users/ojonay/workspace/soccer_odds_env/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


Let's take a look at the first 5 rows of the data:

In [2]:
raw_odds_data.head(5)

,AC,AF,AR,AS,AST,AY,AwayTeam,B365A,B365D,B365H,...,SBH,SJA,SJD,SJH,VCA,VCD,VCH,WHA,WHD,WHH
0,7.0,15.0,0.0,12.0,2.0,2.0,West Ham,4.0,3.30,2.00,...,1.90,3.75,3.4,2.00,4.20,3.25,2.00,4.33,3.25,1.91
1,3.0,14.0,0.0,17.0,12.0,1.0,Everton,2.5,3.25,2.88,...,2.70,2.50,3.3,2.75,2.38,3.30,3.10,2.50,3.20,2.88
2,8.0,13.0,0.0,12.0,7.0,3.0,Fulham,3.4,3.30,2.20,...,2.10,3.00,3.4,2.30,3.40,3.30,2.20,3.50,3.20,2.15
3,1.0,10.0,0.0,10.0,4.0,0.0,West Brom,17.0,7.00,1.17,...,1.12,15.00,6.5,1.18,19.00,7.50,1.17,19.00,6.50,1.17
4,6.0,10.0,0.0,13.0,7.0,3.0,Birmingham,3.6,3.30,2.10,...,2.20,3.20,3.4,2.20,3.50,3.25,2.20,3.50,3.20,2.15


As you can see, we have alot of data to work with. We only need the team names, the odds and the result.
For the odds, we will use the maximum betting odds from betbrain.com (columns BbAvH, BbAvA and BbAvD for maximum home odds, maximum away odds and maximum draw odds.
As for the result, we will use the the FTR (Full Time Result) column, and change the values from (H, A, D) to (0, 1, 2).

In [3]:
odds_dataframe = raw_odds_data[['HomeTeam', 'AwayTeam', 'BbAvH', 'BbAvD', 'BbAvA']]
odds_dataframe.rename({'HomeTeam' :'home_team', 'AwayTeam': 'away_team', 'BbAvH': 'home',
                       'BbAvA': 'away', 'BbAvD': 'draw'}, axis='columns', inplace=True)
odds_dataframe['result'] = raw_odds_data['FTR'].map({'H': 0, 'A': 2, 'D': 1})
odds_dataframe.dropna(subset=['result'], inplace=True)

/Users/ojonay/workspace/soccer_odds_env/lib/python3.6/site-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)
/Users/ojonay/workspace/soccer_odds_env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/ojonay/workspace/soccer_odds_env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation

Let's take a look at the first 5 rows of our odds dataframe:

In [4]:
odds_dataframe.head(5)

,home_team,away_team,home,draw,away,result
0,Aston Villa,West Ham,1.96,3.30,4.03,0.0
1,Blackburn,Everton,2.92,3.25,2.44,0.0
2,Bolton,Fulham,2.20,3.26,3.32,1.0
3,Chelsea,West Brom,1.16,6.90,17.47,0.0
4,Sunderland,Birmingham,2.18,3.25,3.39,1.0


Now we can use our custom environment and pass to it our odds dataframe:

In [5]:
from oddsgym.envs.soccer import ThreeWaySoccerOddsEnv
env = ThreeWaySoccerOddsEnv(odds_dataframe)
max_steps_limit = odds_dataframe.shape[0]
print(max_steps_limit)
print(env._results)

2660
[0. 0. 1. ... 2. 0. 2.]


Let's see what happens when we only bet on the home team:

In [6]:
env.reset()
for _ in range(1, max_steps_limit):
    print(env.render())
    obs, reward, done, truncated, info = env.step(1)
    if done:
        break

Home Team Aston Villa VS Away Team West Ham. Current balance at step 0: 10
Home Team Blackburn VS Away Team Everton. Current balance at step 1: 10.96
Home Team Bolton VS Away Team Fulham. Current balance at step 2: 12.88
Home Team Chelsea VS Away Team West Brom. Current balance at step 3: 11.88
Home Team Sunderland VS Away Team Birmingham. Current balance at step 4: 12.040000000000001
Home Team Tottenham VS Away Team Man City. Current balance at step 5: 11.040000000000001
Home Team Wigan VS Away Team Blackpool. Current balance at step 6: 10.040000000000001
Home Team Wolves VS Away Team Stoke. Current balance at step 7: 9.040000000000001
Home Team Liverpool VS Away Team Arsenal. Current balance at step 8: 10.370000000000001
Home Team Man United VS Away Team Newcastle. Current balance at step 9: 9.370000000000001
Home Team Arsenal VS Away Team Blackpool. Current balance at step 10: 9.600000000000001
Home Team Birmingham VS Away Team Blackburn. Current balance at step 11: 9.76000000000000

We can see that after ~50 games, we strike out. Let's try a random gambler:

In [7]:
env.reset()
for _ in range(1, max_steps_limit):
    print(env.render())
    obs, reward, done, truncated, info = env.step(env.action_space.sample())
    if done:
        break

Home Team Aston Villa VS Away Team West Ham. Current balance at step 0: 10
Home Team Blackburn VS Away Team Everton. Current balance at step 1: 9.0
Home Team Bolton VS Away Team Fulham. Current balance at step 2: 10.92
Home Team Chelsea VS Away Team West Brom. Current balance at step 3: 9.92
Home Team Sunderland VS Away Team Birmingham. Current balance at step 4: 9.08
Home Team Tottenham VS Away Team Man City. Current balance at step 5: 7.08
Home Team Wigan VS Away Team Blackpool. Current balance at step 6: 7.359999999999999
Home Team Wolves VS Away Team Stoke. Current balance at step 7: 6.359999999999999
Home Team Liverpool VS Away Team Arsenal. Current balance at step 8: 5.6899999999999995
Home Team Man United VS Away Team Newcastle. Current balance at step 9: 5.93
Home Team Arsenal VS Away Team Blackpool. Current balance at step 10: 4.16
Home Team Birmingham VS Away Team Blackburn. Current balance at step 11: 3.16
Home Team Everton VS Away Team Wolves. Current balance at step 12: 3.

In [8]:
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [9]:
learning_env = DummyVecEnv([lambda: env])
model = PPO2(MlpPolicy, learning_env, verbose=1)
obs = learning_env.reset()
print(model.action_probability(obs))
model.learn(total_timesteps=2500)





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



[[0.12561612 0.12450422 0.12542842 0.12543969 0.12405596 0.12527315
  0.12446998 0.12521243]]


/Users/ojonay/workspace/soccer_odds_env/lib/python3.6/site-packages/stable_baselines/ppo2/ppo2.py:269: RuntimeWarning: invalid value encountered in subtract
  advs = (advs - advs.mean()) / (advs.std() + 1e-8)
/Users/ojonay/workspace/soccer_odds_env/lib/python3.6/site-packages/numpy/core/_methods.py:193: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


---------------------------------
| approxkl           | nan      |
| clipfrac           | 0.0      |
| explained_variance | nan      |
| fps                | 209      |
| n_updates          | 1        |
| policy_entropy     | nan      |
| policy_loss        | nan      |
| serial_timesteps   | 128      |
| time_elapsed       | 1.91e-06 |
| total_timesteps    | 128      |
| value_loss         | nan      |
---------------------------------
---------------------------------
| approxkl           | nan      |
| clipfrac           | 0.0      |
| explained_variance | nan      |
| fps                | 299      |
| n_updates          | 2        |
| policy_entropy     | nan      |
| policy_loss        | nan      |
| serial_timesteps   | 256      |
| time_elapsed       | 0.612    |
| total_timesteps    | 256      |
| value_loss         | nan      |
---------------------------------
---------------------------------
| approxkl           | nan      |
| clipfrac           | 0.0      |
| explained_va

In [10]:
test_odds_data = pd.read_csv('http://www.football-data.co.uk/mmz4281/1718/E0.csv')
test_odds_data['Date'] = pd.to_datetime(test_odds_data['Date'], dayfirst=True)
test_odds_dataframe = test_odds_data[['HomeTeam', 'AwayTeam', 'BbMxH', 'BbMxD', 'BbMxA']]
test_odds_dataframe.rename({'HomeTeam' :'home_team', 'AwayTeam': 'away_team', 'BbMxH': 'home',
                       'BbMxA': 'away', 'BbMxD': 'draw'}, axis='columns', inplace=True)
test_odds_dataframe['result'] = test_odds_data['FTR'].map({'H': 0, 'A': 2, 'D': 1})
test_odds_dataframe.dropna(subset=['result'], inplace=True)

/Users/ojonay/workspace/soccer_odds_env/lib/python3.6/site-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)
/Users/ojonay/workspace/soccer_odds_env/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/ojonay/workspace/soccer_odds_env/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-doc

In [11]:
env = ThreeWaySoccerOddsEnv(test_odds_dataframe)
max_steps_limit = test_odds_dataframe.shape[0]
print(max_steps_limit)
print(env._results)

380
[0 2 2 2 0 1 1 0 0 2 2 2 0 0 0 0 2 0 2 1 2 2 1 0 0 1 0 0 1 1 0 0 2 2 0 2 1
 0 2 0 0 2 1 1 0 1 2 1 1 0 1 0 2 0 2 2 2 2 0 0 1 2 2 0 0 1 0 0 2 1 1 0 1 0
 0 0 0 1 1 1 2 0 0 0 0 0 2 2 2 0 0 2 1 0 0 2 2 1 0 0 0 2 2 1 2 2 0 0 0 0 0
 0 0 1 2 0 0 2 0 1 1 0 1 0 2 1 1 2 2 0 1 0 2 1 0 2 0 0 0 2 2 2 0 0 0 0 1 1
 1 0 0 1 0 2 0 0 0 1 2 1 0 0 2 1 0 2 2 2 0 1 0 1 0 2 0 2 2 2 2 0 1 0 2 1 1
 0 1 0 1 2 1 0 1 0 1 0 0 1 2 2 0 0 1 0 1 1 2 1 1 1 2 2 0 2 0 2 2 0 1 1 1 0
 2 1 0 1 0 0 0 0 0 2 2 1 0 0 0 1 1 0 2 0 1 2 0 0 1 1 1 0 0 0 0 1 1 0 2 1 1
 0 0 0 1 0 0 0 0 0 2 0 1 0 1 1 0 0 2 2 0 2 0 1 0 1 0 0 0 0 0 2 0 0 1 0 0 2
 2 0 2 2 0 2 0 2 2 2 2 0 0 1 2 0 0 2 1 1 1 2 2 2 2 1 0 1 0 0 0 0 2 1 2 2 0
 1 1 2 2 1 1 1 0 0 1 0 1 0 2 1 2 0 2 0 2 0 0 0 1 2 2 0 0 0 0 1 2 1 0 0 0 1
 2 0 2 0 0 0 2 2 0 0]


In [12]:
testing_env = env
obs = testing_env.reset()
print(testing_env.render())
for i in range(1, max_steps_limit):
    print(obs)
    action, _states = model.predict(obs)
    print(testing_env._verbose_actions[action])
    obs, reward, done, _ = testing_env.step(action)
    print(reward)
    print(testing_env.render())
    if done:
        break

Home Team Arsenal VS Away Team Leicester. Current balance at step 0: 10
      0    1     2
0  1.55  4.6  6.89
[()]
0.0
Home Team Brighton VS Away Team Man City. Current balance at step 1: 10.0
      0    1     2
0  11.5  5.6  1.36
[()]
0.0
Home Team Chelsea VS Away Team Burnley. Current balance at step 2: 10.0
      0     1     2
0  1.27  6.55  15.5
[()]
0.0
Home Team Crystal Palace VS Away Team Huddersfield. Current balance at step 3: 10.0
      0     1     2
0  1.86  3.65  5.11
[()]
0.0
Home Team Everton VS Away Team Stoke. Current balance at step 4: 10.0
      0     1    2
0  1.71  3.85  6.0
[()]
0.0
Home Team Southampton VS Away Team Swansea. Current balance at step 5: 10.0
      0     1    2
0  1.66  4.05  6.5
[()]
0.0
Home Team Watford VS Away Team Liverpool. Current balance at step 6: 10.0
     0    1     2
0  6.5  4.3  1.65
[()]
0.0
Home Team West Brom VS Away Team Bournemouth. Current balance at step 7: 10.0
     0    1    2
0  2.5  3.3  3.3
[()]
0.0
Home Team Man United VS Aw

0.0
Home Team Leicester VS Away Team West Brom. Current balance at step 79: 10.0
      0    1    2
0  2.13  3.5  4.3
[()]
0.0
Home Team West Ham VS Away Team Brighton. Current balance at step 80: 10.0
      0     1     2
0  1.87  3.55  5.43
[()]
0.0
Home Team Chelsea VS Away Team Watford. Current balance at step 81: 10.0
     0    1     2
0  1.4  5.5  10.0
[()]
0.0
Home Team Huddersfield VS Away Team Man United. Current balance at step 82: 10.0
       0    1     2
0  11.73  5.3  1.37
[()]
0.0
Home Team Man City VS Away Team Burnley. Current balance at step 83: 10.0
      0     1     2
0  1.15  12.4  31.0
[()]
0.0
Home Team Newcastle VS Away Team Crystal Palace. Current balance at step 84: 10.0
      0    1     2
0  2.06  3.6  4.33
[()]
0.0
Home Team Southampton VS Away Team West Brom. Current balance at step 85: 10.0
     0    1    2
0  1.9  3.6  5.5
[()]
0.0
Home Team Stoke VS Away Team Bournemouth. Current balance at step 86: 10.0
      0    1    2
0  2.23  3.5  3.8
[()]
0.0
Home Tea

0  1.25  7.25  15.25
[()]
0.0
Home Team Man United VS Away Team Bournemouth. Current balance at step 164: 10.0
     0     1     2
0  1.3  6.35  13.0
[()]
0.0
Home Team Newcastle VS Away Team Everton. Current balance at step 165: 10.0
     0    1     2
0  2.6  3.4  3.35
[()]
0.0
Home Team Southampton VS Away Team Leicester. Current balance at step 166: 10.0
      0    1     2
0  2.12  3.5  4.18
[()]
0.0
Home Team Swansea VS Away Team Man City. Current balance at step 167: 10.0
      0    1     2
0  15.0  7.0  1.27
[()]
0.0
Home Team Tottenham VS Away Team Brighton. Current balance at step 168: 10.0
      0    1     2
0  1.27  6.8  16.0
[()]
0.0
Home Team West Ham VS Away Team Arsenal. Current balance at step 169: 10.0
     0     1     2
0  5.5  4.45  1.67
[()]
0.0
Home Team Arsenal VS Away Team Newcastle. Current balance at step 170: 10.0
      0    1     2
0  1.29  6.8  15.0
[()]
0.0
Home Team Brighton VS Away Team Burnley. Current balance at step 171: 10.0
     0    1    2
0  2.7  3.3

[()]
0.0
Home Team Man City VS Away Team Newcastle. Current balance at step 235: 10.0
      0     1     2
0  1.15  11.0  26.0
[()]
0.0
Home Team Stoke VS Away Team Huddersfield. Current balance at step 236: 10.0
     0     1     2
0  2.1  3.45  4.23
[()]
0.0
Home Team West Ham VS Away Team Bournemouth. Current balance at step 237: 10.0
     0    1    2
0  2.1  3.7  4.0
[()]
0.0
Home Team Southampton VS Away Team Tottenham. Current balance at step 238: 10.0
      0     1    2
0  5.75  4.14  1.7
[()]
0.0
Home Team Swansea VS Away Team Liverpool. Current balance at step 239: 10.0
      0    1     2
0  12.0  5.8  1.35
[()]
0.0
Home Team Huddersfield VS Away Team Liverpool. Current balance at step 240: 10.0
      0     1    2
0  12.0  5.75  1.4
[()]
0.0
Home Team Swansea VS Away Team Arsenal. Current balance at step 241: 10.0
     0     1     2
0  7.5  4.62  1.55
[()]
0.0
Home Team West Ham VS Away Team Crystal Palace. Current balance at step 242: 10.0
      0     1     2
0  3.11  3.32  2.6

0.0
Home Team Watford VS Away Team Bournemouth. Current balance at step 309: 10.0
      0     1     2
0  2.42  3.51  3.27
[()]
0.0
Home Team West Brom VS Away Team Burnley. Current balance at step 310: 10.0
      0    1     2
0  2.71  3.2  3.16
[()]
0.0
Home Team West Ham VS Away Team Southampton. Current balance at step 311: 10.0
      0    1    2
0  2.99  3.3  2.8
[()]
0.0
Home Team Arsenal VS Away Team Stoke. Current balance at step 312: 10.0
      0     1     2
0  1.35  6.15  11.0
[()]
0.0
Home Team Chelsea VS Away Team Tottenham. Current balance at step 313: 10.0
     0     1     2
0  2.5  3.41  3.26
[()]
0.0
Home Team Bournemouth VS Away Team Crystal Palace. Current balance at step 314: 10.0
      0     1     2
0  2.53  3.51  3.15
[()]
0.0
Home Team Brighton VS Away Team Huddersfield. Current balance at step 315: 10.0
      0    1    2
0  1.92  3.6  5.2
[()]
0.0
Home Team Everton VS Away Team Liverpool. Current balance at step 316: 10.0
      0     1     2
0  4.33  3.76  1.99
[()

In [13]:
testing_env = env
results = []
for j in range(100):
    obs = testing_env.reset()
    for i in range(1, max_steps_limit):
        action, _states = model.predict(obs)
        obs, reward, done, _ = testing_env.step(action)
        if done:
            break
    results.append(testing_env.balance)

In [14]:
import numpy

In [15]:
results = numpy.array(results)

In [16]:
results.max(), results.mean()

(10.0, 10.0)